# Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

### Import dataset

In [92]:
walsdata = pd.read_csv('language.csv', sep=',')
# walsdata = walsdata.reindex(sorted(walsdata.columns), axis=1)

walsdata_mgsm = walsdata.loc[walsdata['wals_code'].isin(['eng','ben','mnd','fre','ger','jpn','rus','spa','swa','tel','tha'])]
# walsdata_xcopa = walsdata.loc[walsdata['wals_code'].isin(['eng','est','hcr','ind','que','tml','tur','vie','mnd','swa','tha'])]

In [115]:
# walsdata.iloc[:,130:140]
walsdata.loc[walsdata['wals_code'] == "eng"]


,wals_code,iso_code,glottocode,Name,latitude,longitude,genus,family,macroarea,countrycodes,1A Consonant Inventories,2A Vowel Quality Inventories,3A Consonant-Vowel Ratio,4A Voicing in Plosives and Fricatives,5A Voicing and Gaps in Plosive Systems,6A Uvular Consonants,7A Glottalized Consonants,8A Lateral Consonants,9A The Velar Nasal,10A Vowel Nasalization,11A Front Rounded Vowels,12A Syllable Structure,13A Tone,14A Fixed Stress Locations,15A Weight-Sensitive Stress,16A Weight Factors in Weight-Sensitive Stress Systems,17A Rhythm Types,18A Absence of Common Consonants,19A Presence of Uncommon Consonants,20A Fusion of Selected Inflectional Formatives,21A Exponence of Selected Inflectional Formatives,22A Inflectional Synthesis of the Verb,23A Locus of Marking in the Clause,24A Locus of Marking in Possessive Noun Phrases,25A Locus of Marking: Whole-language Typology,26A Prefixing vs. Suffixing in Inflectional Morphology,27A Reduplication,28A Case Syncretism,29A Syncretism in Verbal Person/Number Marking,30A Number of Genders,31A Sex-based and Non-sex-based Gender Systems,32A Systems of Gender Assignment,33A Coding of Nominal Plurality,34A Occurrence of Nominal Plurality,35A Plurality in Independent Personal Pronouns,36A The Associative Plural,37A Definite Articles,38A Indefinite Articles,39A Inclusive/Exclusive Distinction in Independent Pronouns,40A Inclusive/Exclusive Distinction in Verbal Inflection,41A Distance Contrasts in Demonstratives,42A Pronominal and Adnominal Demonstratives,43A Third Person Pronouns and Demonstratives,44A Gender Distinctions in Independent Personal Pronouns,45A Politeness Distinctions in Pronouns,46A Indefinite Pronouns,47A Intensifiers and Reflexive Pronouns,48A Person Marking on Adpositions,49A Number of Cases,50A Asymmetrical Case-Marking,51A Position of Case Affixes,52A Comitatives and Instrumentals,53A Ordinal Numerals,54A Distributive Numerals,55A Numeral Classifiers,56A Conjunctions and Universal Quantifiers,57A Position of Pronominal Possessive Affixes,58A Obligatory Possessive Inflection,59A Possessive Classification,"60A Genitives, Adjectives and Relative Clauses",61A Adjectives without Nouns,62A Action Nominal Constructions,63A Noun Phrase Conjunction,64A Nominal and Verbal Conjunction,65A Perfective/Imperfective Aspect,66A The Past Tense,67A The Future Tense,68A The Perfect,69A Position of Tense-Aspect Affixes,70A The Morphological Imperative,71A The Prohibitive,72A Imperative-Hortative Systems,73A The Optative,74A Situational Possibility,75A Epistemic Possibility,76A Overlap between Situational and Epistemic Modal Marking,77A Semantic Distinctions of Evidentiality,78A Coding of Evidentiality,79A Suppletion According to Tense and Aspect,80A Verbal Number and Suppletion,"81A Order of Subject, Object and Verb",82A Order of Subject and Verb,83A Order of Object and Verb,"84A Order of Object, Oblique, and Verb",85A Order of Adposition and Noun Phrase,86A Order of Genitive and Noun,87A Order of Adjective and Noun,88A Order of Demonstrative and Noun,89A Order of Numeral and Noun,90A Order of Relative Clause and Noun,91A Order of Degree Word and Adjective,92A Position of Polar Question Particles,93A Position of Interrogative Phrases in Content Questions,94A Order of Adverbial Subordinator and Clause,95A Relationship between the Order of Object and Verb and the Order of Adposition and Noun Phrase,96A Relationship between the Order of Object and Verb and the Order of Relative Clause and Noun,97A Relationship between the Order of Object and Verb and the Order of Adjective and Noun,98A Alignment of Case Marking of Full Noun Phrases,99A Alignment of Case Marking of Pronouns,100A Alignment of Verbal Person Marking,101A Expression of Pronominal Subjects,102A Verbal Person Marking,103A Third Person Zero of Verbal Person Marking,104A Order of Person Markers on the Verb,105A Ditransitive Constructions: The Verb 'Give',106A Reciprocal Constructions,107A Passive Constructions,108A Antipassive Constructions,109

In [113]:
def get_wals_data(lang):
    """
    Get the wals dataframe row of a specific language.
    
    Parameters:
    lang: wals_code of the language of interest.
    
    Returns:
    pd.series row of the wals data for the language.
    """
    return walsdata.loc[walsdata['wals_code'] == lang] # .values.flatten().tolist() get list of walsdata row for a requested language
   
def calc_distance(lang1,lang2):
    """
    Calculate the Haversine distance between the geographical locations of two 
    languages given the wals_codes.
    
    Parameters:
    lang1, lang2: wals_code of two languages to be compared.
    
    Returns:
    Rounded distance between the geographical location of the languages in kilometers.
    """

    # get df for both languages
    wals_lang1 = get_wals_data(lang1)
    wals_lang2 = get_wals_data(lang2)
    
    lat1 = wals_lang1['latitude']
    lon1 = wals_lang1['longitude']
    lat2 = wals_lang2['latitude']
    lon2 = wals_lang2['longitude']

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of the Earth (mean value in kilometers)
    radius_earth = 6371

    # Calculate the distance
    distance = radius_earth * c

    return round(distance)

# calc_distance('eng','dut')

def get_distance_df(langlist):
    """
    Calculate the Haversine distance between the geographical locations of a 
    list of languages with the wals_codes and creates a df with the pairwise
    distances.
    
    Parameters:
    langlist: list with wals_codes of languages to be compared.
    
    Returns:
    Rounded distance between the geographical location of the languages in kilometers.
    """
    df = pd.DataFrame(index=langlist, columns=langlist)

    for lang1 in langlist:
        for lang2 in langlist:
            # print(lang1,lang2,calc_distance(lang1,lang2))
            df.loc[lang1,lang2] = calc_distance(lang1,lang2)

    return df

def get_family_sim_df(langlist):
    """
    Create a df of with similarity based on family. The similarity 
    is a binary value. 
    
    Parameters:
    langlist: list with wals_codes of languages to be compared.
    
    Returns:
    Df with binary value between each language.
    """
    df = pd.DataFrame(index=langlist, columns=langlist)

    for lang1 in langlist:
        for lang2 in langlist:

            # get df for both languages
            wals_lang1 = get_wals_data(lang1)
            wals_lang2 = get_wals_data(lang2)

            value = 1 if wals_lang1.at[wals_lang1.index[0],'family'] == wals_lang2.at[wals_lang2.index[0],'family'] else 0
                      
            df.loc[lang1,lang2] = value

    return df

def get_genus_sim_df(langlist):
    """
    Create a df of with similarity based on genus. The similarity 
    is a binary value. 
    
    Parameters:
    langlist: list with wals_codes of languages to be compared.
    
    Returns:
    Df with binary value between each language.
    """
    df = pd.DataFrame(index=langlist, columns=langlist)

    for lang1 in langlist:
        for lang2 in langlist:

            # get df for both languages
            wals_lang1 = get_wals_data(lang1)
            wals_lang2 = get_wals_data(lang2)

            value = 1 if wals_lang1.at[wals_lang1.index[0],'genus'] == wals_lang2.at[wals_lang2.index[0],'genus'] else 0
                      
            df.loc[lang1,lang2] = value

    return df

def get_macroarea_sim_df(langlist):
    """
    Create a df of with similarity based on macroarea. The similarity 
    is a binary value. 
    
    Parameters:
    langlist: list with wals_codes of languages to be compared.
    
    Returns:
    Df with binary value between each language.
    """
    df = pd.DataFrame(index=langlist, columns=langlist)

    for lang1 in langlist:
        for lang2 in langlist:

            # get df for both languages
            wals_lang1 = get_wals_data(lang1)
            wals_lang2 = get_wals_data(lang2)

            value = 1 if wals_lang1.at[wals_lang1.index[0],'macroarea'] == wals_lang2.at[wals_lang2.index[0],'macroarea'] else 0
                      
            df.loc[lang1,lang2] = value

    return df

##################################### how to deal with nan's? ##############################

def get_sim_df(langlist,featurelist):
    """
    Create a df of similarity between a list of languages based on feature set. The similarity 
    is a sum of binary value (1: feature values are equal for the language pair, 0: else). 
    
    Parameters:
    langlist: list with wals_codes of languages to be compared.
    
    Returns:
    Df with summed binary value between each language.
    """
    df = pd.DataFrame(index=langlist, columns=langlist)
    df = df.fillna(value=0)

    for lang1 in langlist:

        for lang2 in langlist:

            # get df for both languages
            wals_lang1 = get_wals_data(lang1)
            wals_lang2 = get_wals_data(lang2)
            
            for feature in featurelist:

                if pd.notna(wals_lang1.at[wals_lang1.index[0], feature]) and pd.notna(wals_lang2.at[wals_lang2.index[0], feature]): # only check equality for language features that are not nan
                    
                    value = 1 if wals_lang1.at[wals_lang1.index[0],feature] == wals_lang2.at[wals_lang2.index[0],feature] else 0 # set 1 for equal value, 0 for unequal

                    df.loc[lang1,lang2] += value
            # if lang1 != lang2:
            #     df.loc[lang1,lang2] = round(df.loc[lang1,lang2] / df.loc[lang1,lang1],2)
    
    return df 

In [114]:
### specify languages of interest
languages_mgsm = ['eng','spa','fre','swa', 'chi', 'jpn','ben','ger','rus','tel','tha'] #mgsm languages
languages_llama2 = ['eng', 'ukr', 'kor', 'ger', 'fre', 'swe', 'chi', 'spa', 'rus', 'dut', 'ita', 'jpn', 'pol', 'por', 'vie', 'cat', 'ser', 'ind', 'cze', 'fin', 'hun', 'nor', 'rom', 'bul', 'dan', 'slo', 'cro'] #llama-2 languages

languages = ['eng','dut','fri','afr']
# languages = ['eng','ben','tel','hin']


### feature groups
genus = ['genus']
family = ['family']
phonology = ['1A Consonant Inventories','2A Vowel Quality Inventories','3A Consonant-Vowel Ratio','4A Voicing in Plosives and Fricatives','5A Voicing and Gaps in Plosive Systems','6A Uvular Consonants','7A Glottalized Consonants','8A Lateral Consonants','9A The Velar Nasal','10A Vowel Nasalization','11A Front Rounded Vowels','12A Syllable Structure','13A Tone','14A Fixed Stress Locations','15A Weight-Sensitive Stress','16A Weight Factors in Weight-Sensitive Stress Systems','17A Rhythm Types','18A Absence of Common Consonants','19A Presence of Uncommon Consonants']
morphology = ['20A Fusion of Selected Inflectional Formatives','21A Exponence of Selected Inflectional Formatives','22A Inflectional Synthesis of the Verb','23A Locus of Marking in the Clause','24A Locus of Marking in Possessive Noun Phrases','25A Locus of Marking: Whole-language Typology','26A Prefixing vs. Suffixing in Inflectional Morphology','27A Reduplication','28A Case Syncretism','29A Syncretism in Verbal Person/Number Marking']
nominal_cat = ['30A Number of Genders','31A Sex-based and Non-sex-based Gender Systems','32A Systems of Gender Assignment','33A Coding of Nominal Plurality','34A Occurrence of Nominal Plurality','35A Plurality in Independent Personal Pronouns','36A The Associative Plural','37A Definite Articles','38A Indefinite Articles','39A Inclusive/Exclusive Distinction in Independent Pronouns','40A Inclusive/Exclusive Distinction in Verbal Inflection','41A Distance Contrasts in Demonstratives','42A Pronominal and Adnominal Demonstratives','43A Third Person Pronouns and Demonstratives','44A Gender Distinctions in Independent Personal Pronouns','45A Politeness Distinctions in Pronouns','46A Indefinite Pronouns','47A Intensifiers and Reflexive Pronouns','48A Person Marking on Adpositions','49A Number of Cases','50A Asymmetrical Case-Marking','51A Position of Case Affixes','52A Comitatives and Instrumentals','53A Ordinal Numerals','54A Distributive Numerals','55A Numeral Classifiers','56A Conjunctions and Universal Quantifiers','57A Position of Pronominal Possessive Affixes']
nominal_syn = ['58A Obligatory Possessive Inflection','59A Possessive Classification','60A Genitives, Adjectives and Relative Clauses','61A Adjectives without Nouns','62A Action Nominal Constructions','63A Noun Phrase Conjunction','64A Nominal and Verbal Conjunction']
verbal_cat = ['65A Perfective/Imperfective Aspect','66A The Past Tense','67A The Future Tense','68A The Perfect','69A Position of Tense-Aspect Affixes','70A The Morphological Imperative','71A The Prohibitive','72A Imperative-Hortative Systems','73A The Optative','74A Situational Possibility','75A Epistemic Possibility','76A Overlap between Situational and Epistemic Modal Marking','77A Semantic Distinctions of Evidentiality','78A Coding of Evidentiality','79A Suppletion According to Tense and Aspect','80A Verbal Number and Suppletion']
wordorder = ['81A Order of Subject, Object and Verb','82A Order of Subject and Verb','83A Order of Object and Verb','84A Order of Object, Oblique, and Verb','85A Order of Adposition and Noun Phrase','86A Order of Genitive and Noun','87A Order of Adjective and Noun','88A Order of Demonstrative and Noun','89A Order of Numeral and Noun','90A Order of Relative Clause and Noun','91A Order of Degree Word and Adjective','92A Position of Polar Question Particles','93A Position of Interrogative Phrases in Content Questions','94A Order of Adverbial Subordinator and Clause','95A Relationship between the Order of Object and Verb and the Order of Adposition and Noun Phrase','96A Relationship between the Order of Object and Verb and the Order of Relative Clause and Noun','97A Relationship between the Order of Object and Verb and the Order of Adjective and Noun','143A Order of Negative Morpheme and Verb','143B Obligatory Double Negation','143C Optional Double Negation','143D Optional Triple Negation','143E Preverbal Negative Morphemes','143F Postverbal Negative Morphemes','143G Minor morphological means of signaling negation','144A Position of Negative Word With Respect to Subject, Object, and Verb','144B Position of negative words relative to beginning and end of clause and with respect to adjacency to verb','144C Languages with different word order in negative clauses','144D The Position of Negative Morphemes in SVO Languages','144E Multiple Negative Constructions in SVO Languages','144F Obligatory Double Negation in SVO languages','144G Optional Double Negation in SVO languages','144H NegSVO Order','144I SNegVO Order','144J SVNegO Order','144K SVONeg Order','144L The Position of Negative Morphemes in SOV Languages','144M Multiple Negative Constructions in SOV Languages','144N Obligatory Double Negation in SOV languages','144O Optional Double Negation in SOV languages','144P NegSOV Order','144Q SNegOV Order','144R SONegV Order','144S SOVNeg Order','144T The Position of Negative Morphemes in Verb-Initial Languages','144U Double negation in verb-initial languages','144V Verb-Initial with Preverbal Negative','144W Verb-Initial with Negative that is Immediately Postverbal or between Subject and Object','144X Verb-Initial with Clause-Final Negative','144Y The Position of Negative Morphemes in Object-Initial Languages']
simple_clause = ["100A Alignment of Verbal Person Marking","101A Expression of Pronominal Subjects","102A Verbal Person Marking","103A Third Person Zero of Verbal Person Marking","104A Order of Person Markers on the Verb","105A Ditransitive Constructions: The Verb 'Give'","106A Reciprocal Constructions","107A Passive Constructions","108A Antipassive Constructions","108B Productivity of the Antipassive Construction","109A Applicative Constructions","109B Other Roles of Applied Objects","110A Periphrastic Causative Constructions","111A Nonperiphrastic Causative Constructions","112A Negative Morphemes","113A Symmetric and Asymmetric Standard Negation","114A Subtypes of Asymmetric Standard Negation","115A Negative Indefinite Pronouns and Predicate Negation","116A Polar Questions","117A Predicative Possession","118A Predicative Adjectives","119A Nominal and Locational Predication","120A Zero Copula for Predicate Nominals","121A Comparative Constructions"]
complex_sentences = ["122A Relativization on Subjects","123A Relativization on Obliques","124A 'Want' Complement Subjects","125A Purpose Clauses","126A 'When' Clauses","127A Reason Clauses","128A Utterance Complement Clauses"]
lexicon = ['129A Hand and Arm','130A Finger and Hand','131A Numeral Bases','132A Number of Non-Derived Basic Colour Categories','133A Number of Basic Colour Categories','134A Green and Blue','135A Red and Yellow','136A M-T Pronouns','137A N-M Pronouns','138A Tea']

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
get_sim_df(languages,genus+family+phonology+morphology+nominal_cat+nominal_syn+verbal_cat+wordorder+simple_clause+complex_sentences+lexicon)

,eng,dut,fri,afr
eng,151,56,16,13
dut,56,87,20,9
fri,16,20,24,3
afr,13,9,3,18


In [67]:
languages = walsdata.loc[walsdata['genus'] == 'Slavic']
# languages = walsdata.loc[walsdata['family'] == 'Tai-Kadai']
languages


# Swahili: Bantoid, Niger-Congo (Zulu?)
# English, German, Dutch, Frisian, Afrikaans, Swedish, Danish, Norwegian: Germanic, Indo-European
# Spanish, French, Portugese, Italian: Romance, Indo-European

# Russian, Czech, Polish, Ukrainian: Slavic, Indo-European

# Telugu: South-Central Dravidian, Dravidian > Tamil (Dravidian)
# Bengali, Hindi, Maithili, Marathi, Nepali: Indic, Indo-European 
# Thai: Kam-Tai, Tai-Kadai (Lao?)
# Mandarin, Cantonese: Chinese, Sino-Tibetan

# Japanese: Japanese, Japanese

walsdata.loc[walsdata['Name'] == 'Afrikaans']

,wals_code,iso_code,glottocode,Name,latitude,longitude,genus,family,macroarea,countrycodes,...,137B M in Second Person Singular,136B M in First Person Singular,109B Other Roles of Applied Objects,10B Nasal Vowels in West Africa,25B Zero Marking of A and P Arguments,21B Exponence of Tense-Aspect-Mood Inflection,108B Productivity of the Antipassive Construction,130B Cultural Categories of Languages with Identity of 'Finger' and 'Hand',58B Number of Possessive Nouns,79B Suppletion in Imperatives and Hortatives
36,afr,afr,afri1274,Afrikaans,-31.0,22.0,Germanic,Indo-European,Africa,ZA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
